In [1]:
# pip install pandas, datasets, pylance

import numpy as np
import pandas as pd
from datasets import load_dataset

NUM_SAMPLES_BUFFER = 100000 # to not overload your disk
NUM_SAMPLES = 10000

# Login using e.g. `huggingface-cli login` to access this dataset
# 37gb i dont have -vlad --> streaming mode = true
raw_dataset = load_dataset("bigscience-data/roots_code_stackexchange", split="train", streaming=True)
shuffled_dataset = raw_dataset.shuffle(buffer_size=NUM_SAMPLES_BUFFER)

c:\Users\vladk\miniconda3\envs\snlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
shuffled_dataset.features

{'text': Value(dtype='string', id=None),
 'meta': Value(dtype='string', id=None)}

In [24]:
batch = next(iter(shuffled_dataset.batch(NUM_SAMPLES)))
series = pd.Series(batch["text"], dtype=str)
series

0       Q:\n\nFFMPEG - Store FFPROBE's duration output...
1       Q:\n\nAngular 6 Asp.Net (not Core) Web Api COR...
2       Q:\n\nHow do I package extension functions in ...
3       Q:\n\nHTML | CSS: Issues with hamburger menu\n...
4       Q:\n\nIterate data to Pandas dataframe (python...
                              ...                        
9995    Q:\n\nWhy for loop is not working in ionic ion...
9996    Q:\n\nMAX DateTime value in JavaScript\n\nIn ....
9997    Q:\n\ntest iterator value in struts 2\n\nI wan...
9998    Q:\n\nHow to create a drop down in React?\n\nI...
9999    Q:\n\nReplacing fill values in 2D array with v...
Length: 10000, dtype: object

In [9]:
Q_SEG = "Q:\n\n"
A_SEG = "\n\nA:\n\n"

a = series[series.str.contains(Q_SEG)]
b = a[a.str.contains(A_SEG)]
b

0       Q:\n\nFFMPEG - Store FFPROBE's duration output...
1       Q:\n\nAngular 6 Asp.Net (not Core) Web Api COR...
2       Q:\n\nHow do I package extension functions in ...
3       Q:\n\nHTML | CSS: Issues with hamburger menu\n...
4       Q:\n\nIterate data to Pandas dataframe (python...
                              ...                        
9995    Q:\n\nWhy for loop is not working in ionic ion...
9996    Q:\n\nMAX DateTime value in JavaScript\n\nIn ....
9997    Q:\n\ntest iterator value in struts 2\n\nI wan...
9998    Q:\n\nHow to create a drop down in React?\n\nI...
9999    Q:\n\nReplacing fill values in 2D array with v...
Length: 10000, dtype: object

In [23]:
def split_qna(series: pd.Series):
    split = series.str.strip(Q_SEG).str.split(A_SEG, expand=True)[[0, 1]]
    # ^^ btw there are more than 1 answers per question for some entries
    split.columns = ["question", "answer"]
    split["full_text"] = series # "full_text" will reflect this reality as well
    return split

c = split_qna(b)
c

,question,answer,full_text
0,FFMPEG - Store FFPROBE's duration output as a ...,The command designed for processing output of ...,Q:\n\nFFMPEG - Store FFPROBE's duration output...
1,Angular 6 Asp.Net (not Core) Web Api CORS requ...,After struggling my head the whole morning and...,Q:\n\nAngular 6 Asp.Net (not Core) Web Api COR...
2,How do I package extension functions in a libr...,"Silly mistake, the extension functions are vi...",Q:\n\nHow do I package extension functions in ...
3,HTML | CSS: Issues with hamburger menu\n\nMy n...,"Ok, I think I've found a solution to your prob...",Q:\n\nHTML | CSS: Issues with hamburger menu\n...
4,Iterate data to Pandas dataframe (python)\n\nI...,"Use:\nimport requests, pandas, numpy, matplotl...",Q:\n\nIterate data to Pandas dataframe (python...
...,...,...,...
9995,Why for loop is not working in ionic ionViewWi...,ionViewWillEnter() gets for a less time and he...,Q:\n\nWhy for loop is not working in ionic ion...
9996,"MAX DateTime value in JavaScript\n\nIn .NET, t...",You can use this to get the JS Date object:\nn...,Q:\n\nMAX DateTime value in JavaScript\n\nIn ....
9997,test iterator value in struts 2\n\nI want to s...,Have you checked that it is not a simple typo?...,Q:\n\ntest iterator value in struts 2\n\nI wan...
9998,How to create a drop down in React?\n\nI am tr...,validateSalary(e){\n // e.persist();\n ...,Q:\n\nHow to create a drop down in React?\n\nI...
